<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Определение количества кластеров</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.datasets import make_classification

%matplotlib inline

In [ ]:
from sklearn.metrics import (
    silhouette_score, 
    calinski_harabasz_score, 
    davies_bouldin_score,
    adjusted_rand_score,
    adjusted_mutual_info_score,
    fowlkes_mallows_score
)

In [ ]:
from matplotlib.colors import ListedColormap
clrMap = ListedColormap(["blue", "red", "green", "yellow", "purple", "orange"])

<h3><b>1. Формирование начальных данных</b></h3>

<p>Формируем набор данных из 500 элементов с тремя признаками и 5-ю классами (один кластер на один класс)</p>

In [ ]:
NUM_SAMPLES = 500
RANDOM_STATE = 12345

In [ ]:
X, y = make_classification(n_samples=NUM_SAMPLES, n_features=3, n_redundant=0,
                           n_informative=3, n_clusters_per_class=1, n_classes=5, 
                           class_sep=2,random_state=RANDOM_STATE)

<p>Отображаем исходные данные без указания кластеров</p>

In [ ]:
plt.figure(figsize=[15, 5])

plt.subplot(1,3,1)
plt.title("X1-X2")
plt.xlabel("X1")
plt.ylabel("X2")
plt.scatter(X[:,0], X[:,1])
plt.grid(True)

plt.subplot(1,3,2)
plt.title("X1-X3")
plt.xlabel("X1")
plt.ylabel("X3")
plt.scatter(X[:,0], X[:,2])
plt.grid(True)

plt.subplot(1,3,3)
plt.title("X2-X3")
plt.xlabel("X2")
plt.ylabel("X3")
plt.scatter(X[:,1], X[:,2])
plt.grid(True)

plt.tight_layout()

plt.show()

<p>Отображаем исходные данные с исходными (действительными) кластерами</p>

In [ ]:
plt.figure(figsize=[15, 5])

plt.subplot(1,3,1)
plt.title("X1-X2")
plt.xlabel("X1")
plt.ylabel("X2")
plt.scatter(X[:,0], X[:,1], c=y, cmap=clrMap)
plt.grid(True)

plt.subplot(1,3,2)
plt.title("X1-X3")
plt.xlabel("X1")
plt.ylabel("X3")
plt.scatter(X[:,0], X[:,2], c=y, cmap=clrMap)
plt.grid(True)

plt.subplot(1,3,3)
plt.title("X2-X3")
plt.xlabel("X2")
plt.ylabel("X3")
plt.scatter(X[:,1], X[:,2], c=y, cmap=clrMap)
plt.grid(True)

plt.tight_layout()

plt.show()

<h3><b>2. Поиск количества кластеров</b></h3>

<p>В качестве метода кластеризации будем использовать k-means со следующими параметрами:</p>
<ul>
    <li>максимальное количество итераций (max_iter) - 300</li>
    <li>способ задания начального положения кластеров (init) - k-means++ </li>
    <li>количество повторных запусков алгоритма (n_init) - 20 - для поиска наилучшего разбиения данных на заданное количество кластеров (критерий выбора - разбиение с наименьшим показателем инерции)</li>
    <li>количество кластеров (n_clusters) - регулируем</li>
</ul>

<p><b>A. Показатель инерции</b></p>

<p>Определяем показатель инерции для различного количества кластеров</p>

In [ ]:
clusters = np.arange(2, 16)
inert = np.full(clusters.size, 0)

for i in range(clusters.size):
    model = KMeans(n_clusters=clusters[i], max_iter=300, init="k-means++", random_state=RANDOM_STATE, n_init=20)
    model.fit(X)
    inert[i] = model.inertia_

<p>Строим график</p>

In [ ]:
plt.title("NumClusters-Inertia")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.plot(clusters, inert, "-o")
plt.grid(True)
plt.show()

<p>Определяем максимальное отношение между разностями инерции соседних значений количества кластеров</p>

In [ ]:
def predict_cluster_index(inertias):
    max_value = 0.0
    indx = 1
    for i in range(1, len(inertias)-1):
        cur_value = (inertias[i-1]-inertias[i]) / (inertias[i]-inertias[i+1])
        if cur_value > max_value:
            max_value = cur_value
            indx = i
    return indx

In [ ]:
num_clusters__inertia = clusters[predict_cluster_index(inert)]
num_clusters__inertia

<p><b>B. Коэффициент Silhouette</b></p>

<p>Вычисляем коэффициенты Silhouette</p>

In [ ]:
silhouette_scores = np.full(clusters.size, 0.0)
for i in range(clusters.size):
    model = KMeans(n_clusters=clusters[i], max_iter=300, init="k-means++", n_init=20, random_state=RANDOM_STATE)
    model.fit(X)
    labels_pred = model.labels_
    silhouette_scores[i] = silhouette_score(X, labels_pred, metric="euclidean")

<p>Строим график</p>

In [ ]:
plt.title("NumClusters-Silhouette Coefficient")
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette Coefficient")
plt.plot(clusters, silhouette_scores, "-o")
plt.grid(True)
plt.show()

<p>Определяем наилучшее количество кластеров</p>

In [ ]:
num_clusters__silhouette = clusters[silhouette_scores.argmax()]
num_clusters__silhouette

<p><b>B. Индекс Calinski-Harabaz</b></p>

<p>Вычисляем индекс Calinski-Harabaz</p>

In [ ]:
calinski_scores = np.full(clusters.size, 0.0)

for i in range(clusters.size):
    model = KMeans(n_clusters=clusters[i], max_iter=300, init="k-means++", n_init=20, random_state=RANDOM_STATE)
    model.fit(X)
    labels_pred = model.labels_
    calinski_scores[i] = calinski_harabasz_score(X, labels_pred)

<p>Строим график</p>

In [ ]:
plt.title("NumClusters-Calinski-Harabaz Index")
plt.xlabel("Number of clusters")
plt.ylabel("Calinski-Harabaz Index")
plt.plot(clusters, calinski_scores, "-o")
plt.grid(True)
plt.show()

<p>Определяем наилучшее количество кластеров</p>

In [ ]:
num_clusters__calinski = clusters[calinski_scores.argmax()]
num_clusters__calinski

<h3><b>3. Анализ полученных результатов с учетом знания о действительном количестве кластеров</b></h3>

<p>Проверяемые значения: 5 и 7 кластеров</p>

<p>Определяем модели для соответствующего количества кластеров</p>

In [ ]:
model__num_5 = KMeans(n_clusters=5, max_iter=300, init="k-means++", random_state=10, n_init=20)
model__num_7 = KMeans(n_clusters=7, max_iter=300, init="k-means++", random_state=10, n_init=20)

<p>Запускаем поиск (обучение модели) кластеров</p>

In [ ]:
labels_pred__num_5 = model__num_5.fit(X).labels_
labels_pred__num_5[:10]

In [ ]:
labels_pred__num_7 = model__num_7.fit(X).labels_
labels_pred__num_7[:10]

<p>Используем метрики для сравнения полученных (предсказанных) результатов с действительными значениям (полученными при формировании начальных данных)</p>

<p><b>A. Adjusted Rand index</b></p>

In [ ]:
# для 5 кластеров
adjusted_rand_score(y, labels_pred__num_5) # от -1.0 до 1.0. -1.0 - плохо, 1.0 - хорошо

In [ ]:
# для 7 кластеров
adjusted_rand_score(y, labels_pred__num_7) 

<p><b>B. Mutual Information based scores</b></p>

In [ ]:
# для 5 кластеров
adjusted_mutual_info_score(y, labels_pred__num_5, 
                                   average_method="arithmetic") # от 0 до 1. 0 - плохо, 1.0 - хорошо

In [ ]:
# для 7 кластеров
adjusted_mutual_info_score(y, labels_pred__num_7, 
                                   average_method="arithmetic") 

<p><b>C. Fowlkes-Mallows scores</b></p>

In [ ]:
# для 5 кластеров
fowlkes_mallows_score(y, labels_pred__num_5) # от 0 до 1. 0 - плохо, 1.0 - хорошо

In [ ]:
# для 7 кластеров
fowlkes_mallows_score(y, labels_pred__num_7) 

## Кластеризация изображений цифр

In [ ]:
from sklearn import datasets

In [ ]:
RANDOM_STATE = 12345

In [ ]:
# Загрузка исходных данных
digits = datasets.load_digits()

IMAGE_INDX = 3

# Отображение одного изображения
print("Image:")
plt.imshow(digits.images[IMAGE_INDX])
plt.show()
print("Feature matrix:\n", digits.images[IMAGE_INDX])
print("\nTarget value:", digits.target[IMAGE_INDX])

In [ ]:
# Преобразование исходных данных
# Замечание: digits.data уже содержит преобразованные данные
X = digits.images.reshape(len(digits.images), -1)
labels_true = digits.target

Подбор количества кластеров

In [ ]:
num_clusters = np.arange(2, 17)

metrics = [
    ("Inertia", None),
    ("Silhouette Coefficient", silhouette_score),
    ("Calinski-Harabasz Index", calinski_harabasz_score),
    ("Davies-Bouldin Index", davies_bouldin_score)
]

scores = [
    np.full(len(num_clusters), float("inf")),
    np.full(len(num_clusters), 0.0),
    np.full(len(num_clusters), float("inf")),
    np.full(len(num_clusters), float("inf"))
]

for indx, num in enumerate(num_clusters):
    model = KMeans(n_clusters=num, max_iter=300, init="k-means++", n_init=20, random_state=RANDOM_STATE)
    model.fit(X)
    scores[0][indx] = model.inertia_
    for j in range(1, len(metrics)):
        scores[j][indx] = metrics[j][1](X, model.labels_)

Отображение графиков

In [ ]:
num_metrics = len(metrics)
num_cmlns = 3
num_rows = int(np.ceil(num_metrics/num_cmlns))

plt.figure(figsize=[18, 4*num_rows])

for row_indx in range(num_rows):
    for clmn_indx in range(num_cmlns):
        indx = num_cmlns*row_indx + clmn_indx
        if indx == num_metrics:
            break
        plt.subplot(num_rows, num_cmlns, indx + 1)
        plt.title(metrics[indx][0])
        plt.xlabel("Number of clusters")
        plt.ylabel(metrics[indx][0])
        plt.plot(num_clusters, scores[indx], "-o")
        plt.grid(True)

plt.tight_layout()
plt.show()

Количество кластеров по инерции

In [ ]:
num_clusters[predict_cluster_index(scores[0])]

Обучение

In [ ]:
NUM_CLUSTERS = 9
NUM_DISPLAY_IMAGES = 10

In [ ]:
# Кластеризация
model = KMeans(n_clusters=NUM_CLUSTERS, max_iter=300, init="k-means++", n_init=20, random_state=RANDOM_STATE)
model.fit(X)

# Выявленные кластеры
labels_pred = model.labels_

for label in range(NUM_CLUSTERS):

    # Индексы элементов кластера label
    labels_pred__indices = np.asarray(labels_pred==label).nonzero()[0]
    
    # Выбираем случайным образом 10 индексов элементов кластера label
    np.random.seed(RANDOM_STATE)
    labels_pred___indices_ = np.random.choice(labels_pred__indices, NUM_DISPLAY_IMAGES, replace=False)
    
    # Отображения выбранных элементов кластера
    print("Cluster label:", label)
    plt.figure(figsize=[14, 4])
    for i in range(NUM_DISPLAY_IMAGES):
        plt.subplot(1, NUM_DISPLAY_IMAGES, i+1)
        plt.title(labels_pred___indices_[i])
        plt.imshow(digits.images[labels_pred___indices_[i]])
        plt.axis("off")
    plt.show()

In [ ]:
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
cm = contingency_matrix(labels_true, labels_pred)
cm = np.c_[cm, np.zeros(cm.shape[0])]
cm

In [ ]:
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=np.array(list(range(NUM_CLUSTERS + 1))),
)
disp.plot(values_format='1g')
plt.title('Contingency matrix')
plt.show()

<h3><b>4. Источники</b></h3>

<a href="http://scikit-learn.org/stable/modules/clustering.html">Clustering</a><br>
<a href="http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans">sklearn.cluster.KMeans</a>